# 下载模型

In [3]:
!gdown https://drive.google.com/uc?id=1-5v7gCPWaKKBtJVu0KZwvlDyEia40crr

Downloading...
From: https://drive.google.com/uc?id=1-5v7gCPWaKKBtJVu0KZwvlDyEia40crr
To: /content/RE-BERT/demo_api_8apps_v1.model
100% 455M/455M [00:05<00:00, 83.9MB/s]


# 克隆仓库

In [ ]:
!git clone https://github.com/adailtonaraujo/RE-BERT


In [1]:
# 更新仓库
%cd RE-BERT
!git pull

/content/RE-BERT
Already up to date.


# 安装依赖

In [2]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 加载模型

In [3]:

from re_bert_demo import re_bert_model
from re_bert_demo import extract
import numpy as np

options = ['--classifier_model_file','demo_api_8apps_v1.model']
model = re_bert_model(options)  # 加载模型

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
loading model RE_BERT ...


# 用例1

> I use the app to apply filters to my photos and it's really fun.

In [ ]:
review = "I use the app to apply filters to my photos and it's really fun."
predictions,requirements_extracted = extract(model,review)


print('\n\nReview: ',review)
print('Software Requirements Extracted (in each sentence):',requirements_extracted)

for item in predictions:
  sentence = item[0]
  software_requirements = item[1]
  iob_classification = item[2]
  print('\n\nSentence:',sentence)
  print('Software Requirements Tokens:',software_requirements)
  for tokens_res in iob_classification:
    token = tokens_res[0]
    confidence = np.max(tokens_res[1]['confidences'])
    iob_class = np.max(tokens_res[1]['iob'])
    iob_tag = 'O'
    if iob_class == 0: iob_tag='B'
    if iob_class == 1: iob_tag='I'
    print('Token [',token,'] is tag [',iob_tag,'] with',confidence,'confidence.')

# 用例2

> I am trying to set up a business account and change my email address and I am having the most difficulty using your app its totally ridiculous! And another thing I am unable to see prices until driver has been found.

In [ ]:
review = "I am trying to set up a business account and change my email address and I am having the most difficulty using your app its totally ridiculous! And another thing I am unable to see prices until driver has been found."
predictions,requirements_extracted = extract(model,review)


print('\n\nReview: ',review)
print('Software Requirements Extracted (in each sentence):',requirements_extracted)

for item in predictions:
  sentence = item[0]
  software_requirements = item[1]
  iob_classification = item[2]
  print('\n\nSentence:',sentence)
  print('Software Requirements Tokens:',software_requirements)
  for tokens_res in iob_classification:
    token = tokens_res[0]
    confidence = np.max(tokens_res[1]['confidences'])
    iob_class = np.max(tokens_res[1]['iob'])
    iob_tag = 'O'
    if iob_class == 0: iob_tag='B'
    if iob_class == 1: iob_tag='I'
    print('Token [',token,'] is tag [',iob_tag,'] with',confidence,'confidence.')

## 模型评估

**生成标注文件**

In [4]:
!python extract.py --test_file datasets_iob/test_data_Twitter.txt --classifier_model_file demo_api_8apps_v1.model --output_file Twitter_extracted_reqs.txt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
loading model RE_BERT ...
Traceback (most recent call last):
  File "extract.py", line 232, in <module>
    main()
  File "extract.py", line 190, in main
    f = open(opt.test_file, 'r')
FileNotFoundError: [Errno 2] No such file or directory: 'test_data_Twitter.txt'


**与人工标注比对**

In [ ]:
reviews_test = open('datasets_iob/test_data_Twitter.txt', 'r').readlines()
reqs_exracted =  open('Twitter_extracted_reqs.txt', 'r').readlines()
reqs_labeled =  open('datasets_iob/test_label_Twitter.txt', 'r').readlines()

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

df = pd.DataFrame()
df['review'] = reviews_test
df['labeled (humans)'] = reqs_labeled
df['extracted (re-bert)'] = reqs_exracted
df = df.replace('\n',' ', regex=True)
df